# Esercitazione SemEval

In [1]:
import hashlib
import re
import csv
import os.path
import os

from scipy.stats import pearsonr, spearmanr
from scipy.spatial.distance import cosine
from sklearn.metrics import cohen_kappa_score
from py_babelnet.calls import BabelnetAPI

KEY = '5ed5ed15-4bb7-4968-9d1d-eac164a4f95f'
#KEY = 'b02f0cb7-4143-4e23-a581-6ff8b54bab88'
babelnet = BabelnetAPI(KEY)

### Consegna 1 - Annotazione di coppie di parole 

L'annotazione consiste in un punteggio di semantic similarity di 50 coppie di termini.

Il criterio da utilizzare è presente al link https://tinyurl.com/y6f8h2kd. 

In particolare: 
- 4: **Molto simili** -- Le parole sono sinonimi (e.g., midday-noon). 
- 3: **Simili** -- Le parole condividono gran parte delle idee di significato ma includono dettagli profondamente differenti (e.g., lion-zebra). 
- 2: **Leggermente simili** -- Le parole non hanno significato molto simile ma condividono un argomento/dominio/funzione/idee/concetti che sono correlati (e.g., house-window). 
- 1: **Differenti** -- Le parole descrivono chiaramente concetti differenti, ma condividono qualche piccolo dettaglio come una lontana relazione o un dominio di utilizzo simile in un documento (e.g., software-keyboard).
- 0: **Totalmente differenti e scorrelati** -- Le due parole non significano la stessa cosa e riguardano argomenti differenti (e.g., pencil-frog).

1.1 Estrazione di 50 coppie a partire dal cognome "Scarpinati"

In [2]:
def get_range(surname: str) -> int:
    nof_elements = 500
    base_idx = (abs(int(hashlib.sha512(surname.encode('utf-8')).hexdigest(), 16)) % 10)
    idx_intervallo = base_idx * 50+1
    return idx_intervallo
 

input_name = "Scarpinati"

values = []
sx = get_range(input_name)
values.append(sx)
dx = sx+50-1
intervallo = "" + str(sx) + "-" + str(dx)
print('{:15}:\tcoppie nell\'intervallo {}'.format(input_name, intervallo))


Scarpinati     :	coppie nell'intervallo 251-300


In [3]:
path_corpus = f"utils/it.test.data.txt"

with open(path_corpus, 'r', encoding='utf8') as file:
    reader = file.readlines()[sx:dx+1]
    pairs = [line.strip().split("\t") for line in reader]
    print(pairs)
    print(len(pairs))

[['crittogramma', 'simbolo'], ['campus', 'università'], ['libertà', 'libertà'], ['tifone', 'ciclone'], ['manuale', 'guida turistica'], ['Ebola', 'virus Ebola'], ['principessa', 'biscotto'], ['treno', 'tram'], ['ecosistema', 'economia'], ['regina regnante', 'regina degli scacchi'], ['oro', 'zinco'], ['cursore', 'patatine fritte'], ['Rio delle Amazzoni', 'foresta'], ['tennis', 'statistica'], ['Bolzano', 'teorema'], ['altitudine', 'conversione'], ['barca', 'albero'], ['trono', 'spada'], ['cotone', 'maglione'], ['ciliegia', 'fragola'], ['Islam', 'Corano'], ['Neanderthal', 'sport'], ['nepotismo', 're'], ['personaggio', 'persona'], ['cisterna', 'vagone'], ['Mercurio', 'Giove'], ['bronchite', 'acetaminofene'], ['sangue', 'corpo'], ['islamofobia', 'ISIS'], ['uncinetto', 'uniforme'], ['Hadoop', 'touchscreen'], ['combustibile fossile', 'fossile'], ['mutante', 'sociobiologia'], ['sorella', 'fratello'], ['matita', 'storia'], ['eczema', 'dermatite'], ['imperatore', 'governatore'], ['corona', 'chiav

1.2 Memorizzazione delle coppie estratte

**ATTENZIONE!** NON ESEGUIRE PIU' QUESTA CELLA, IL FILE E' GIA' STATO CREATO

In [4]:
path_extracted = f"resources/it.test.data_extracted.tsv"
if not os.path.exists(path_extracted):
    with open(path_extracted, 'w', newline='', encoding='utf8') as out_file:
        tsv_writer = csv.writer(out_file, delimiter='\t')
        tsv_writer.writerow(["Term1", "Term2"])
        tsv_writer.writerows(pairs)

1.3 Estrazione dei valori di similarità inseriti per ogni coppia e calcolo della media

In [5]:
similarity_livio = []
similarity_laura = []
mean_similarity = []

path_annotated = f"resources/part1/it.test.data_annotated.tsv"

if os.path.exists(path_annotated):
    with open(path_annotated, 'r', encoding='utf8') as file:
        tsv_file = csv.reader(file, delimiter="\t")
        header = next(tsv_file)
        for line in tsv_file:
            similarity_livio.append(float(line[2]))
            similarity_laura.append(float(line[3]))

mean_similarity = [(similarity_livio[i] + similarity_laura[i])/2 for i in range(50)]

print(similarity_livio)
print(similarity_laura)
print(mean_similarity)

print(len(similarity_laura))
print(len(similarity_livio))
print(len(mean_similarity))

[3.0, 3.0, 4.0, 4.0, 2.0, 4.0, 0.0, 3.0, 2.0, 2.0, 3.0, 0.0, 3.0, 0.0, 0.0, 0.0, 1.0, 3.0, 3.0, 3.0, 2.0, 0.0, 2.0, 2.0, 0.0, 3.0, 1.0, 2.0, 1.0, 1.0, 0.0, 0.0, 0.0, 3.0, 0.0, 2.0, 3.0, 0.0, 1.0, 1.0, 0.0, 3.0, 3.0, 0.0, 0.0, 1.0, 0.0, 3.0, 3.0, 2.0]
[2.5, 3.2, 4.0, 4.0, 3.0, 3.5, 0.2, 3.5, 2.3, 0.0, 3.2, 0.0, 2.5, 0.5, 0.0, 0.0, 1.2, 2.5, 3.2, 3.3, 3.0, 0.2, 2.2, 3.2, 1.2, 3.2, 1.5, 3.3, 1.2, 1.3, 0.1, 2.2, 0.1, 3.6, 0.3, 2.7, 3.3, 0.2, 3.0, 3.0, 1.2, 3.7, 3.4, 3.0, 1.3, 1.2, 1.0, 3.2, 3.7, 1.5]
[2.75, 3.1, 4.0, 4.0, 2.5, 3.75, 0.1, 3.25, 2.15, 1.0, 3.1, 0.0, 2.75, 0.25, 0.0, 0.0, 1.1, 2.75, 3.1, 3.15, 2.5, 0.1, 2.1, 2.6, 0.6, 3.1, 1.25, 2.65, 1.1, 1.15, 0.05, 1.1, 0.05, 3.3, 0.15, 2.35, 3.15, 0.1, 2.0, 2.0, 0.6, 3.35, 3.2, 1.5, 0.65, 1.1, 0.5, 3.1, 3.35, 1.75]
50
50
50


1.4 Memorizzazione del documento con il valore medio di valutazione per ogni coppia

In [6]:
os.remove(path_annotated)

with open(path_annotated, 'w', newline='', encoding='utf8') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    tsv_writer.writerow(["Term1", "Term2", "Sim_Livio", "Sim_Laura", "Mean"])
    for pair, sim_livio, sim_laura, mean in zip(pairs, similarity_livio, similarity_laura, mean_similarity):
        tsv_writer.writerow([pair[0], pair[1], sim_livio, sim_laura, mean])

1.5 Calcolare agreement fra gli annotatori (inter-rater agreement)

Vengono utilizzati gli indici di correlazione di **Pearson** e **Spearman**

In [7]:
print("##### Inter Agreement with Pearson & Spearman:")
pearson = pearsonr(similarity_livio, similarity_laura)
spearman = spearmanr(similarity_livio, similarity_laura)
print("Pearson ")
print("Coefficiente di correlazione:", pearson[0])
print("p-value: ", pearson[1])
print("\n\nSpearman ")
print("Coefficiente di correlazione:", spearman.correlation)
print("p-value: ", spearman.pvalue)

##### Inter Agreement with Pearson & Spearman:
Pearson 
Coefficiente di correlazione: 0.824495945350208
p-value:  1.8586060579974576e-13


Spearman 
Coefficiente di correlazione: 0.8401818319529221
p-value:  2.3703924660488e-14


1.6 Valutazione

La valutazione dei punteggi annotati è in rapporto alla similarità ottenuta utilizzando i vettori NASARI. 

In [8]:
def get_nasari_vectors() -> dict: 
    path_nasari = f"utils/mini_NASARI.tsv"
    nasari_vectors = {}
    with open(path_nasari, 'r', encoding='utf8') as file:
        tsv_file = csv.reader(file, delimiter="\t")
        for line in tsv_file:
            synset_id, synset_name = line[0].split("__")
            vector = [float(value) for value in line[1:]]
            nasari_vectors[synset_id] = vector
    return nasari_vectors

def get_senses2synsets() -> dict: 
    path_senses2synsets = f"utils/SemEval17_IT_senses2synsets.txt"
    senses2synsets = {}
    with open(path_senses2synsets, 'r', encoding='utf8') as file:
        reader = file.readlines()
        i = 0
        while i < len(reader):
            if reader[i].startswith("#"):
                synset_name = reader[i][1:].strip()
                synset_ids = []
                i += 1
                while i < len(reader) and not reader[i].startswith("#"):
                    synset_ids.append(reader[i].strip())
                    i += 1
                senses2synsets[synset_name] = synset_ids
    return senses2synsets

def get_synsets(word: str, nasari_vectors: dict, senses2synsets: dict) -> list[tuple[str, list[float]]]:
    synsets = []
    if word in senses2synsets:
        for synset_id in senses2synsets[word]:
            if synset_id in nasari_vectors:
                synsets.append((synset_id, nasari_vectors[synset_id]))
    return synsets


Per il calcolo della similarità è stata utilizzata la *cosine similarity*


In [9]:
def get_cosine_similarity(v1: list, v2: list) -> float:
    return 1 - cosine(v1, v2)


def get_similarity(word1: str, word2: str, nasari_vectors: dict, senses2synsets: dict) -> tuple[float, tuple[str, str]]:
    """
    Restituisce la similarità massima tra i vettori NASARI dei due termini e vettori nasari che la massimizzano, se presenti.
    Altrimenti restituisce 0.
    """
    synsets1 = get_synsets(word1, nasari_vectors, senses2synsets)
    synsets2 = get_synsets(word2, nasari_vectors, senses2synsets)
    max_similarity = 0
    max_similarity_ids = ("", "")
    if len(synsets1) > 0 or len(synsets2) > 0:
        for id1, v1 in synsets1:
            for id2, v2 in synsets2:
                similarity = get_cosine_similarity(v1, v2)
                if similarity > max_similarity:
                    max_similarity = similarity
                    max_similarity_ids = (id1, id2)
                    
    return max_similarity, max_similarity_ids
 

In [10]:
nasari_vectors = get_nasari_vectors()
senses2synsets = get_senses2synsets()

similarities_nasari = []
for word1, word2 in pairs: 
    print(word1, word2)
    similarity, _ = get_similarity(word1, word2, nasari_vectors, senses2synsets)
    similarities_nasari.append(round(similarity, 4))
    print("Similarity:", round(similarity, 4))

crittogramma simbolo
Similarity: 0.624
campus università
Similarity: 0.9408
libertà libertà
Similarity: 1
tifone ciclone
Similarity: 1
manuale guida turistica
Similarity: 0.8345
Ebola virus Ebola
Similarity: 0.993
principessa biscotto
Similarity: 0.3956
treno tram
Similarity: 0.8679
ecosistema economia
Similarity: 0.4969
regina regnante regina degli scacchi
Similarity: 0
oro zinco
Similarity: 0.8507
cursore patatine fritte
Similarity: 0.3544
Rio delle Amazzoni foresta
Similarity: 0.6174
tennis statistica
Similarity: 0.6604
Bolzano teorema
Similarity: 0.5703
altitudine conversione
Similarity: 0.7004
barca albero
Similarity: 0.9279
trono spada
Similarity: 0.5891
cotone maglione
Similarity: 0.6254
ciliegia fragola
Similarity: 0.9073
Islam Corano
Similarity: 0.8843
Neanderthal sport
Similarity: 0.5034
nepotismo re
Similarity: 0.654
personaggio persona
Similarity: 1
cisterna vagone
Similarity: 0.5377
Mercurio Giove
Similarity: 1
bronchite acetaminofene
Similarity: 0.6769
sangue corpo
Simila

Confronto risultati medi dell'annotazione con i risultati ottenuti utilizzando la risorsa NASARI

In [11]:
def normalize_range(values: list, min: int, max: int) -> list:
    """
    Normalizza i valori in [0, 1] a partire dal range di partenza [min, max]
    """
    normalized_values = [(val-min)/(max - min) for val in values]    
    return normalized_values

mean_similarity = normalize_range(mean_similarity, 0, 4)

print("##### word1  word2   norm mean similarity    nasari similarity:\n\n")

for i in range(len(pairs)): 
    print(str(pairs[i]) + "\t" + str(mean_similarity[i]) + "\t\t" + str(similarities_nasari[i]))
    

##### word1  word2   norm mean similarity    nasari similarity:


['crittogramma', 'simbolo']	0.6875		0.624
['campus', 'università']	0.775		0.9408
['libertà', 'libertà']	1.0		1
['tifone', 'ciclone']	1.0		1
['manuale', 'guida turistica']	0.625		0.8345
['Ebola', 'virus Ebola']	0.9375		0.993
['principessa', 'biscotto']	0.025		0.3956
['treno', 'tram']	0.8125		0.8679
['ecosistema', 'economia']	0.5375		0.4969
['regina regnante', 'regina degli scacchi']	0.25		0
['oro', 'zinco']	0.775		0.8507
['cursore', 'patatine fritte']	0.0		0.3544
['Rio delle Amazzoni', 'foresta']	0.6875		0.6174
['tennis', 'statistica']	0.0625		0.6604
['Bolzano', 'teorema']	0.0		0.5703
['altitudine', 'conversione']	0.0		0.7004
['barca', 'albero']	0.275		0.9279
['trono', 'spada']	0.6875		0.5891
['cotone', 'maglione']	0.775		0.6254
['ciliegia', 'fragola']	0.7875		0.9073
['Islam', 'Corano']	0.625		0.8843
['Neanderthal', 'sport']	0.025		0.5034
['nepotismo', 're']	0.525		0.654
['personaggio', 'persona']	0.65		1
['cisterna', 'va

1.7 Coefficienti di correlazione tra la media degli annotatori e i risultati di NASARI

In [12]:
print("##### Intra Agreement with Pearson & Spearman:")

pearson = pearsonr(mean_similarity, similarities_nasari)
spearman = spearmanr(mean_similarity, similarities_nasari)

print("Pearson ")
print("Coefficiente di correlazione:", pearson[0])
print("p-value: ", pearson[1])
print("\n\nSpearman ")
print("Coefficiente di correlazione:", spearman.correlation)
print("p-value: ", spearman.pvalue)

##### Intra Agreement with Pearson & Spearman:
Pearson 
Coefficiente di correlazione: 0.5587957118302855
p-value:  2.469792546476724e-05


Spearman 
Coefficiente di correlazione: 0.6038348992844942
p-value:  3.4431691409166004e-06


### Consegna 2 - Sense Identification

L'obiettivo è di individuare i sensi selezionati utilizzati per generare il giudizio di similarità nella consegna precedente.

La domanda che ci poniamo è la seguente: *quali sensi abbiamo effettivamente utilizzato quando abbiamo assegnato un valore di similarità a una coppia di termini (per esempio, società e cultura)*?

Per risolvere questo compito partiamo dall’assunzione che i due termini funzionino come contesto di disambiguazione l’uno per l’altro.

2.1 Caricamento del file annotato

In [13]:
path_ids_laura = f"resources/part2/it.test.data_ids_laura.tsv"
path_ids_livio = f"resources/part2/it.test.data_ids_livio.tsv"
path_ids_NASARI = f"resources/part2/it.test.data_ids_NASARI.tsv"

ids_laura : list[tuple[str, str]] = []
ids_livio : list[tuple[str, str]] = []

with open(path_ids_laura, 'r', encoding='utf8') as file:
    tsv_file = csv.reader(file, delimiter="\t")
    next(tsv_file)
    for line in tsv_file:
        ids_laura.append((line[2], line[3]))

with open(path_ids_livio, 'r', encoding='utf8') as file:
    tsv_file = csv.reader(file, delimiter="\t")
    next(tsv_file)
    for line in tsv_file:
        ids_livio.append((line[2], line[3]))

In [14]:
def get_terms_synsets(pairs_ids: list[tuple[str, str]]) -> list[tuple[list, list]]:
    """
    Restituisce i termini associati ai synset di ogni coppia di termini a partire dai loro id.
    """
    terms_synsets = []
    for id1, id2 in pairs_ids:
        terms_word1 = set()
        terms_word2 = set()
        
        if id1 != "":
            synset_word1 = babelnet.get_synset(id=id1, searchLang="IT", targetLang="IT")
    
            if "senses" in synset_word1:
                for sense in synset_word1['senses']: 
                    terms_word1.add(sense["properties"]["fullLemma"])
        
        if id2 != "":
            synset_word2 = babelnet.get_synset(id=id2, searchLang="IT", targetLang="IT")
            if "senses" in synset_word2:
                for sense in synset_word2['senses']: 
                    terms_word2.add(sense["properties"]["fullLemma"])
        
        terms_synsets.append((list(terms_word1), list(terms_word2)))

    return terms_synsets


2.1.1 Memorizzazione dei termini per ogni synset inserito nell'annotazione

In [15]:
terms_ids_laura = get_terms_synsets(ids_laura)

print(terms_ids_laura[:3])

[(['criptogramma', 'crittogramma', 'crittogrammi'], ['simbolo', 'segno', 'simbologia', 'simboli', 'simbolico']), (['cittadella_universitaria', 'Campus_universitario', 'città_universitaria', 'Cittadella_universitaria', 'campus_universitario', 'campus'], ['ateneo', 'università']), (['autodeterminazione', 'libertà_personale', 'sovranità', 'Frēodōm', 'indipendenza', 'libertà_individuale', 'libertà'], ['autodeterminazione', 'libertà_personale', 'sovranità', 'Frēodōm', 'indipendenza', 'libertà_individuale', 'libertà'])]


In [16]:
terms_ids_livio = get_terms_synsets(ids_livio)

print(terms_ids_livio[:3])

[(['criptogramma', 'crittogramma', 'crittogrammi'], ['simbolo', 'segno', 'simbologia', 'simboli', 'simbolico']), (['cittadella_universitaria', 'Campus_universitario', 'città_universitaria', 'Cittadella_universitaria', 'campus_universitario', 'campus'], ['ateneo', 'università']), (['autodeterminazione', 'libertà_personale', 'sovranità', 'Frēodōm', 'indipendenza', 'libertà_individuale', 'libertà'], ['autodeterminazione', 'libertà_personale', 'sovranità', 'Frēodōm', 'indipendenza', 'libertà_individuale', 'libertà'])]


Aggiornamento file annotati con i termini associati ai synset

In [17]:
os.remove(path_ids_laura)

with open(path_ids_laura, 'w', newline='', encoding='utf8') as file:
    tsv_writer = csv.writer(file, delimiter='\t')
    tsv_writer.writerow(["Term1", "Term2", "BS1", "BS2", "Terms_in_BS1", "Terms_in_BS2"])
    for pair, id_laura, terms_id_laura in zip(pairs, ids_laura, terms_ids_laura):
        terms_id0 = ','.join(terms_id_laura[0][:3])
        terms_id1 = ','.join(terms_id_laura[1][:3])
        tsv_writer.writerow([pair[0], pair[1], id_laura[0], id_laura[1], terms_id0, terms_id1])

In [18]:
os.remove(path_ids_livio)

with open(path_ids_livio, 'w', newline='', encoding='utf8') as file:
    tsv_writer = csv.writer(file, delimiter='\t')
    tsv_writer.writerow(["Term1", "Term2", "BS1", "BS2", "Terms_in_BS1", "Terms_in_BS2"])
    for pair, id_livio, terms_id_livio in zip(pairs, ids_livio, terms_ids_livio):
        terms_id0 = ','.join(terms_id_livio[0][:3])
        terms_id1 = ','.join(terms_id_livio[1][:3])
        tsv_writer.writerow([pair[0], pair[1], id_livio[0], id_livio[1], terms_id0, terms_id1])

2.2 Coefficiente di correlazione

In [19]:
print("##### Inter Agreement with Cohen Kappa Score:")
ids_laura_id1, ids_laura_id2 = zip(*ids_laura)
ids_livio_id1, ids_livio_id2 = zip(*ids_livio)

kappa_id1 = cohen_kappa_score(ids_laura_id1, ids_livio_id1)
kappa_id2 = cohen_kappa_score(ids_laura_id2, ids_livio_id2)

print("Kappa score for babelsynset id 1:", kappa_id1)
print("Kappa score for babelsynset id 2:", kappa_id2)

##### Inter Agreement with Cohen Kappa Score:
Kappa score for babelsynset id 1: 0.9185004074979625
Kappa score for babelsynset id 2: 0.7562956945572705


2.3 Valutazione annotazione

Vengono estratti i BabelSynsetID che massimizzano la similarità tra le parole e successivamente vengono memorizzati in un file con le parole associate ai synset. 

In [20]:
def get_ids_nasari(pairs: list[tuple[str, str]], nasari_vectors: dict, senses2synsets: dict) -> list[tuple[str, str]]:
    """
    Restituisce i synset associati ai termini di ogni coppia di termini a partire dai vettori NASARI che massimizzano la 
    similarità tra le due parole.
    """
    ids = []
    for word1, word2 in pairs:
        _, ids_synsets = get_similarity(word1, word2, nasari_vectors, senses2synsets)
        ids.append(ids_synsets)
    return ids


In [21]:
ids_nasari = get_ids_nasari(pairs, nasari_vectors, senses2synsets)

terms_ids_nasari = get_terms_synsets(ids_nasari)

print(terms_ids_nasari[:3])

with open(path_ids_NASARI, 'w', newline='', encoding='utf8') as file: 
    tsv_writer = csv.writer(file, delimiter='\t')
    tsv_writer.writerow(["Term1", "Term2", "BS1", "BS2", "Terms_in_BS1", "Terms_in_BS2"])
    for pair, id_nasari, terms_id_nasari in zip(pairs, ids_nasari, terms_ids_nasari):
        terms_id0 = ','.join(terms_id_nasari[0][:3])
        terms_id1 = ','.join(terms_id_nasari[1][:3])
        tsv_writer.writerow([pair[0], pair[1], id_nasari[0], id_nasari[1], terms_id0, terms_id1])



[(['criptogramma', 'crittogramma', 'crittogrammi'], ['semiologia', 'Semiologi', 'semeiologia', 'semiotica', 'semiologo']), (['cittadella_universitaria', 'Campus_universitario', 'città_universitaria', 'Cittadella_universitaria', 'campus_universitario', 'campus'], ['collegio', 'accademia', 'università_di_ricerca', 'università_globale', 'politecnico', 'università', 'status_di_università', 'istituto', 'ateneo', 'università_ricerca']), (['autodeterminazione', 'libertà_personale', 'sovranità', 'Frēodōm', 'indipendenza', 'libertà_individuale', 'libertà'], ['autodeterminazione', 'libertà_personale', 'sovranità', 'Frēodōm', 'indipendenza', 'libertà_individuale', 'libertà'])]


Viene effettuata la valutazione sulla base di quanti id sono diversi nelle valutazioni. 

In [22]:
def get_evaluation_single(ids1: list[str], ids2: list[str]) -> int: 
    score = 0
    for id1, id2 in zip(ids1, ids2):
        if id1 != id2:
            score += 1
    return score

def get_evaluation_multiple(ids1: list[tuple[str, str]], ids2: list[tuple[str, str]]) -> int:
    score = 0
    for id1, id2 in zip(ids1, ids2):
        if id1[0] != id2[0] or id1[1] != id2[1]:
            score += 1
    return score

In [23]:
ids_nasari_id1, ids_nasari_id2 = zip(*ids_nasari)

print("##### Evaluation with Accuracy:")

ev_id1_laura = get_evaluation_single(ids_laura_id1, ids_nasari_id1)
ev_id2_laura = get_evaluation_single(ids_laura_id2, ids_nasari_id2)
ev_id1_livio = get_evaluation_single(ids_livio_id1, ids_nasari_id1)
ev_id2_livio = get_evaluation_single(ids_livio_id2, ids_nasari_id2)

print("Laura's evaluation for babelsynset id 1:", 1 - ev_id1_laura/len(ids_laura_id1))

print("Laura's evaluation for babelsynset id 2:", 1 - ev_id2_laura/len(ids_laura_id2))

print("Livio's evaluation for babelsynset id 1:", 1 - ev_id1_livio/len(ids_livio_id1))

print("Livio's evaluation for babelsynset id 2:", 1 - ev_id2_livio/len(ids_livio_id2))

print("##### Evaluation with Accuracy (multiple):")

ev_ids_laura = get_evaluation_multiple(ids_laura, ids_nasari)
ev_ids_livio = get_evaluation_multiple(ids_livio, ids_nasari)

print("Laura's evaluation for pairs babelsynset:", 1 - ev_ids_laura/len(ids_laura))
print("Livio's evaluation for pairs babelsynset:", 1 - ev_ids_livio/len(ids_livio))

##### Evaluation with Accuracy:
Laura's evaluation for babelsynset id 1: 0.6
Laura's evaluation for babelsynset id 2: 0.45999999999999996
Livio's evaluation for babelsynset id 1: 0.6599999999999999
Livio's evaluation for babelsynset id 2: 0.5
##### Evaluation with Accuracy (multiple):
Laura's evaluation for pairs babelsynset: 0.30000000000000004
Livio's evaluation for pairs babelsynset: 0.33999999999999997
